In [1]:
import pandas as pd
import numpy as np
from dataloading import DataLoader
from eda_py import EDA
from eALS_adaptor import eALSAdaptor
from implicit.evaluation import train_test_split, ranking_metrics_at_k
from cv_py import CrossValidation

%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

C:\Users\781110104\OneDrive - Genpact\Documents\VSCode


In [2]:
dl = DataLoader()

In [3]:
user_item_co = dl.import_data('AGCO', 'CO', 'df')
user_item_co_t = dl.import_data('TEREX', 'CO', 'df')

In [4]:
user_item_filtered = dl.remove_low_interact_items(user_item_co, 1)
user_item_filtered_t = dl.remove_low_interact_items(user_item_co_t, 1)

In [5]:
user_item_filtered_log = dl.log_scale_df(user_item_filtered, 0.01)
user_item_filtered_log_t = dl.log_scale_df(user_item_filtered_t, 0.01)

In [6]:
user_item_csr = dl.to_csr(user_item_filtered_log)
user_item_csr_t = dl.to_csr(user_item_filtered_log_t)

In [7]:
# initial split
train, test = train_test_split(user_item_csr, 0.8, 22)
train_t, test_t = train_test_split(user_item_csr_t, 0.8, 22)

In [8]:
cv = CrossValidation(train, 5)
cv_t = CrossValidation(train_t, 5)

In [9]:
validate, calibrate = cv.split_k_fold(22)
validate_t, calibrate_t = cv_t.split_k_fold(22)

In [13]:
space_iALS = {'factors' : [64], 'regularization' : [30, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160], 'alpha' : [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1], 'iterations' : [15]}
space_iALS_t = {'factors' : [64], 'regularization' : [30, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160], 'alpha' : [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1], 'iterations' : [15]}
hyper_ials = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_iALS, model_class='iALS')
hyper_ials_t = cv.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_iALS_t, model_class='iALS')

In [15]:
hyper_ials.sort_values(by=['precision'], ascending=False)

,factors,regularization,alpha,iterations,precision,map,ndcg,auc,mpr
69,64,90,0.30,15,0.851089,0.800718,0.863477,0.505329,0.087845
92,64,120,0.40,15,0.850495,0.799719,0.863221,0.505365,0.087117
104,64,140,0.50,15,0.849835,0.797916,0.862138,0.505370,0.086530
58,64,80,0.30,15,0.849439,0.797794,0.861797,0.505333,0.089027
80,64,100,0.30,15,0.849109,0.798134,0.861894,0.505326,0.088061
...,...,...,...,...,...,...,...,...,...
66,64,90,0.05,15,0.598152,0.513759,0.633717,0.503216,0.153945
99,64,140,0.05,15,0.462904,0.374296,0.515576,0.502424,0.162283
110,64,160,0.05,15,0.462772,0.373783,0.515551,0.502441,0.162007
88,64,120,0.05,15,0.460858,0.373015,0.513622,0.502403,0.162579


In [16]:
hyper_ials_t.sort_values(by=['precision'], ascending=False)

,factors,regularization,alpha,iterations,precision,map,ndcg,auc,mpr
42,64,60,0.90,15,0.5395,0.423240,0.564148,0.515171,0.124122
65,64,80,1.00,15,0.5390,0.420694,0.566042,0.515249,0.121743
29,64,50,0.70,15,0.5385,0.422086,0.563352,0.515280,0.125842
40,64,60,0.70,15,0.5355,0.419692,0.564796,0.515203,0.122314
54,64,70,1.00,15,0.5345,0.419612,0.561233,0.514882,0.122100
...,...,...,...,...,...,...,...,...,...
100,64,140,0.10,15,0.0450,0.014691,0.043148,0.501212,0.460343
110,64,160,0.05,15,0.0295,0.009418,0.028099,0.500535,0.441363
66,64,90,0.05,15,0.0280,0.013778,0.033017,0.500385,0.436190
55,64,80,0.05,15,0.0235,0.010579,0.027678,0.500341,0.442415
